# Perform spatial queries on points retrieved from TileDB

Note if querying for large areas then you should consider using a larger notebook or breaking the area into smaller queries and using [user defined functions](https://docs.tiledb.com/cloud/client-api/serverless-array-udfs)

In [ ]:
import time

import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
import tiledb

tiledb_uri = 'tiledb://exactEarth/ee_ais'

In [ ]:
%%time

t1 = np.datetime64('2019-07-01T00:00:00')
t2 = np.datetime64('2019-07-02T00:00:00')
x1, x2, y1, y2 = [-1.7, -1.2,  50, 51.0]

st = time.time()

with tiledb.open(tiledb_uri) as arr:
    pts = arr.multi_index[t1:t2, x1:x2, y1:y2]
    
print(f"Retrieved {len(pts['longitude'])} vessels")

We will make a GeoPandas dataframe to perform spatial queries

In [ ]:
df = pd.DataFrame(pts)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs='epsg:4326')

We will quickly inspect the array

In [ ]:
gdf.head()

We will pick one vessel and buffer the location to retrieve all of the vessels nearby

In [ ]:
%%time

pt = gdf.geometry[1000]
thresh = 0.125

bowtie = shapely.geometry.Polygon([
          (pt.x - thresh, pt.y + thresh),
          (pt.x, pt.y),
          (pt.x - thresh, pt.y - thresh),
          (pt.x + thresh, pt.y - thresh),
          (pt.x, pt.y),
          (pt.x + thresh, pt.y + thresh),
          (pt.x - thresh, pt.y + thresh)]).buffer(0) # buffer by zero cleans the intersecting point

r = gpd.clip(gdf, bowtie)

In [ ]:
# note edge of original query in the result dataset
r.plot()

Alternatively use the spatial index directly

In [ ]:
%%time

spatial_index = gdf.sindex
possible_matches_index = list(spatial_index.intersection(bowtie.bounds))
possible_matches = gdf.iloc[possible_matches_index]
precise_matches = possible_matches[possible_matches.intersects(bowtie)]

In [ ]:
precise_matches.plot()

Plot with datashader

In [ ]:
import datashader as ds
from datashader.utils import lnglat_to_meters
import holoviews as hv
import holoviews.operation.datashader as hd
from holoviews.element import tiles
from matplotlib import cm

hv.extension("bokeh", "matplotlib")

In [ ]:
hv.output(backend="bokeh")
r.loc[:, 'x'], r.loc[:, 'y'] = lnglat_to_meters(r.longitude, r.latitude)
bkgrd = tiles.EsriImagery().opts(xaxis=None, yaxis=None, width=700, height=500)
points = hv.Points(r, ['x', 'y'])
bkgrd * hd.datashade(points, cmap=cm.inferno)